In [1]:
!pip install transformers torch pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration
import pandas as pd

In [3]:
file_path = 'artikel_alwi.csv'
df = pd.read_csv(file_path)

In [6]:
df.head()

,Bidang,Judul,Isi
0,Ekonomi,Menko Airlangga Yakin Ekonomi Indonesia Tumbuh...,Pemerintah optimistis ekonomi Indonesia akan t...
1,Ekonomi,Realisasi Investasi Asing Bakal Bantu Pertumbu...,Mandiri Investment Forum 2021 yang akan berlan...
2,Ekonomi,Bank Mandiri Prediksi Ekonomi Indonesia Tumbuh...,PT Bank Mandiri Tbk memprediksi bahwa pertumbu...
3,Ekonomi,"Ekonomi RI Bakal Membaik, intip 3 Sektor Saham...",Pemulihan ekonomi Indonesia diprediksi terjadi...
4,Ekonomi,Vaksinasi Covid-19 Jadi Salah Satu Strategi Pe...,Menteri Perindustrian (Menperin) Agus Gumiwang...


In [8]:
model_name = "facebook/bart-large-cnn"

# Load tokenizer dan model dari pre-trained
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [11]:
summaries = []

# Loop melalui setiap teks dalam kolom 'Isi'
for text in df['Isi']:
    # Tokenisasi teks dan generate ringkasan
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
    summary_ids = model.generate(inputs['input_ids'],
                                 max_length=150,
                                 min_length=30,
                                 length_penalty=2.0,
                                 num_beams=4,
                                 early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)

In [12]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.5 MB/s eta 0:00:00


In [14]:
df['summary'] = summaries

In [15]:
from bert_score import score
# Evaluasi ringkasan menggunakan BERTScore
P, R, F1 = score(df['summary'].tolist(), df['Isi'].tolist(), lang="id", verbose=True)

# Tambahkan skor evaluasi ke dalam dataframe
df['BERTScore_P'] = P.tolist()
df['BERTScore_R'] = R.tolist()
df['BERTScore_F1'] = F1.tolist()

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 104.72 seconds, 0.48 sentences/sec


In [16]:
print("BERTScore Precision:", df['BERTScore_P'])
print("BERTScore Recall:", df['BERTScore_R'])
print("BERTScore F1 Score:", df['BERTScore_F1'])

BERTScore Precision: 0     0.835606
1     0.836351
2     0.855364
3     0.841343
4     0.874421
5     0.894084
6     0.883597
7     0.838399
8     0.856626
9     0.899784
10    0.845942
11    0.846825
12    0.844487
13    0.862476
14    0.854319
15    0.901257
16    0.881577
17    0.895449
18    0.874531
19    0.896095
20    0.898160
21    0.833438
22    0.854907
23    0.874957
24    0.834006
25    0.908963
26    0.849850
27    0.836776
28    0.829850
29    0.855086
30    0.819131
31    0.834751
32    0.873398
33    0.844838
34    0.895674
35    0.835174
36    0.863955
37    0.863004
38    0.832770
39    0.864393
40    0.831821
41    0.857691
42    0.833882
43    0.911469
44    0.844168
45    0.864402
46    0.848424
47    0.851175
48    0.828780
49    0.823562
Name: BERTScore_P, dtype: float64
BERTScore Recall: 0     0.617257
1     0.643392
2     0.660076
3     0.603328
4     0.665729
5     0.699345
6     0.667674
7     0.595350
8     0.648078
9     0.687302
10    0.623549
11    0.6015